In [1]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
import pandas as pd
import numpy as np
import time
import math
import csv
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
from features import *
data_loc = '../data/onepunch/'
pd.set_option('display.max_columns', None)  
from IPython.display import display
import time
import bisect

In [2]:
def readTask(loc = './collate/OrdersDataFrame.csv'):
    with open(loc, 'r') as csvfile:
        raw_queue = pd.read_csv(csvfile, sep=',')
    raw_queue.drop(['time.1','minute'], axis=1, inplace=True)
    raw_queue["time"] = pd.to_datetime(raw_queue["time"], utc=True, errors='coerce')
    raw_queue.index = raw_queue["time"].apply(lambda x: time.mktime(x.timetuple()))
    raw_queue.sort_index(ascending=True, inplace=True)
    #display(raw_queue.head(2))
    return raw_queue

taskData = readTask()
taskData.drop(['time'], axis=1, inplace=True)

In [88]:
new_columns = list(taskqueue)
timeHistKeyList = list(taskqueue.index)
ind = bisect.bisect_left(timeHistKeyList, 1528624790)
print(timeHistKeyList[ind-1])
key = timeHistKeyList[ind-1]
#display(taskqueue.loc[key])
print(new_columns)

1528624620.0
['building_processing', 'complete', 'correcting_labels', 'cv_processing', 'exporting_3d_formats', 'extracting_measurements', 'failed', 'failed_building_processing', 'failed_cv_processing', 'failed_extracting_measurements', 'failed_image_processing', 'failed_primitives_processing', 'failed_pro_measurements_processing', 'failed_zulu_processing', 'image_processing', 'labeling', 'markup_verifying', 'model_segmenting', 'modeling', 'new', 'primitives_processing', 'pro_measurements_processing', 'qa', 'texturing', 'verifying', 'waiting_cv_processing', 'waiting_image_processing', 'waiting_labeling', 'waiting_markup', 'waiting_markup_verifying', 'waiting_model_segmenting', 'waiting_modeling', 'waiting_primitives_processing', 'waiting_pro_measurements_processing', 'waiting_qa', 'waiting_texturing', 'waiting_verifying', 'waiting_web_segmenting', 'waiting_zulu_processing', 'web_segmenting', 'zulu_processing']


In [3]:
def load_data():
    print("Loading Data... ")
    data = {}
    for file_name, column_names in file_column_mappings.items():
        nice_name = file_name_name_mapping.get(file_name, None)
        if nice_name:
            print("Loading %s"%(nice_name))
            file_name = data_loc+file_name
            data[nice_name] = pd.read_csv(file_name,
                                          sep='|',
                                          index_col=False, 
                                          names=column_names,
                                          low_memory=False
                                        )
    print("Data Loaded!")
    return data

raw_data_dic = load_data()
raw_data = raw_data_dic['puborders']

Loading Data... 
Loading puborders
Data Loaded!


In [5]:
def add_task_column(raw_data,taskqueue):
    
    new_columns = list(taskqueue)
    timeHistKeyList = list(taskqueue.index)

    selected_data = raw_data # first add new columns
    for new_name in new_columns:
        selected_data[new_name] = np.nan

    selected_data["order_created_at"] = pd.to_datetime(raw_data["order_created_at"], utc=True, errors='coerce')
    
    
    for index, row in selected_data.iterrows():
        onetime = row['order_created_at']
        timestamp = time.mktime(onetime.timetuple())
        ind = bisect.bisect_left(timeHistKeyList, timestamp) # match with the nearest timestamp
        key = timeHistKeyList[ind-1]
        new_data = taskqueue.loc[key]
        for feed_column in new_columns:
            selected_data[feed_column] = new_data[feed_column] # add the needed data into those places

    return selected_data

selected_data = add_task_column(raw_data,taskData)
display(selected_data.head(2))

ValueError: NaTType does not support timetuple

In [89]:
    
selected_data["order_created_at"] = pd.to_datetime(raw_data["order_created_at"], utc=True, errors='coerce')
for idx, onetime in enumerate(selected_data.order_created_at):
    if idx < 10:
        #print (onetime,time.mktime(onetime.timetuple()))
        timestamp = time.mktime(onetime.timetuple())
        ind = bisect.bisect_left(timeHistKeyList, timestamp)
        key = timeHistKeyList[ind-1]
        new_data = taskqueue.loc[key]
        
display(selected_data.head(1))

(Timestamp('2018-06-10 01:59:50.666520+0000', tz='UTC'), 1528624790.0)
(Timestamp('2018-06-10 18:36:24.225381+0000', tz='UTC'), 1528684584.0)
(Timestamp('2018-06-10 20:26:40.467408+0000', tz='UTC'), 1528691200.0)
(Timestamp('2018-06-11 04:57:14.381157+0000', tz='UTC'), 1528721834.0)
(Timestamp('2018-06-11 10:27:03.158752+0000', tz='UTC'), 1528741623.0)
(Timestamp('2018-06-11 11:22:21.262783+0000', tz='UTC'), 1528744941.0)
(Timestamp('2018-06-11 13:50:10.784184+0000', tz='UTC'), 1528753810.0)
(Timestamp('2018-06-11 14:19:45.696039+0000', tz='UTC'), 1528755585.0)
(Timestamp('2018-06-11 15:46:25.692593+0000', tz='UTC'), 1528760785.0)
(Timestamp('2018-06-11 19:40:34.567596+0000', tz='UTC'), 1528774834.0)


,order_id,order_state,order_created_at,order_updated_at,original_order_id,order_complexity,order_image_score,order_image_score_name,priority,order_deliverable,order_most_recent_failure_reason,site_id,order_local,slayer_job_id,order_site_name,sneaky,order_skip_markups,number_of_stories,e_number_of_stories_name,complexity_attributes_total_roof_facets_above_4_sqft,complexity_attributes_total_roof_facets,complexity_attributes_roof_square_footage,complexity_attributes_walls_square_footage,complexity_attributes_is_mfr,map_attributes_source,slayer_attributes_allows_mfr,slayer_attributes_is_mfr,slayer_attributes_user_first_capture,poser_release_number,structure_type,structure_type_name,poser_solve_status,hover_now,first_qa_datetime,first_qa_date,models_reprocessed,order_needs_to_be_reprocessed_flag,special_request,original_roof,first_qa_for_state,last_qa_for_state,first_qa_email,first_qa_site_name,first_qa_majority_of_doors_within_95_percent_of_standard_sizes,first_qa_map_overlay_accurate,first_qa_scale_within5_percent,first_qa_should_we_unfail_the_order,qa_comments,orthotag_list,level_list,pitch_estimate,roof_estimate,labeler_roof_estimate,markup_verifier_roof_estimate,markup_verifier_10_percent,pitch_actual,scaling_method,visible_on_ortho_map,door_areal_scale_diff,doors_used_for_scaling,door_scale_deviation,orthotag_tree_pass,orthotag_poor_image_quality,orthotag_none,orthotag_full_occlusion,orthotag_not_on_map,orthotag_tree_occlusion,orthotag_tree_fail,orthotag_missing_geometry,orthotag_shadow,poser_solved_image_count,poser_solved,failed_complete_timeline,failed_complete_progression,most_recent_failure_datetime,completed_count,failed_count,failed_or_complete_name_first,failed_or_complete_name_second,failed_or_complete_name_third,failed_or_complete_name_fourth,failed_or_complete_name_fifth,failed_or_complete_datetime_first,failed_or_complete_datetime_second,failed_or_complete_datetime_third,failed_or_complete_datetime_fourth,failed_or_complete_datetime_fifth,number_of_images,average_image_score,image_count_deleted,image_count_0_sides,image_count_1_side,image_count_2_sides,image_count_unscored,first_completion,most_recent_completion,unfail_premium_order_flag,turn_around_time_total,turn_around_time_first_completion,total_resolution_time_total,total_resolution_time_first_completion,human_bpo_time_waiting_total,human_bpo_time_waiting_first_completion,turn_around_time_wasted,human_bpo_time_wasted,labeling_time_total,labeling_time_first_completion,markup_verifying_time_total,markup_verifying_time_first_completion,model_building_time_total,model_building_time_first_completion,modeling_time_total,modeling_time_first_completion,model_completing_time_total,model_completing_time_first_completion,web_segmenting_time_total,web_segmenting_time_first_completion,model_segmenting_time_total,model_segmenting_time_first_completion,verifying_time_total,verifying_time_first_completion,texturing_time_total,texturing_time_first_completion,qa_time_total,openings_marking_time_total,openings_marking_time_first_completion,vps_marking_time_total,vps_marking_time_first_completion,primitive_marking_time_total,primitive_marking_time_first_completion,building_processing,complete,correcting_labels,cv_processing,exporting_3d_formats,extracting_measurements,failed,failed_building_processing,failed_cv_processing,failed_extracting_measurements,failed_image_processing,failed_primitives_processing,failed_pro_measurements_processing,failed_zulu_processing,image_processing,labeling,markup_verifying,model_segmenting,modeling,new,primitives_processing,pro_measurements_processing,qa,texturing,verifying,waiting_cv_processing,waiting_image_processing,waiting_labeling,waiting_markup,waiting_markup_verifying,waiting_model_segmenting,waiting_modeling,waiting_primitives_processing,waiting_pro_measurements_processing,waiting_qa,waiting_texturing,waiting_verifying,waiting_web_segmenting,waiting_zulu_processing,web_segmenting,zulu_processing
0,745942,complete,2018-06-10 01:59:50.666520